# Laboratorio Redes Neuronales


##### Antonio Reyes #17273

Proporcione un modelo de redes neuronales del tipo feed
forward fully connected que prediga el tipo de prenda de vestir es, esto usando todas las
features que considere necesarias

In [1]:
# Importando paquetes
import numpy as np
import pandas as pd
from functools import reduce
from scipy import optimize as op
import pickle
import math

- numpy se utiliza para el manejo de matrices
- pandas se utiliza para la lectura de la base de datos (que se encuentra en un docuemtno .csv
- reduce se utiliza en la función flatten_list_of_arrays para aplanar la matriz con valores theta
- pickle es una librería que se encarga de encriptar un documento texto, se utiliza luego de realizar las iteraciones de back propagation con op (importado de scipy)


In [2]:
#obtener datos de csv
dataset = pd.read_csv('fashion-mnist_train.csv')
X = dataset.iloc[:60000, 1:].values
np.asarray(X)

# tamaño para el dato inicial de las 3 capas de la red neuronal
m,n = X.shape

In [3]:
# valores esperados
Y = np.asarray(dataset.iloc[:60000, 0])

#### X son pixel 1 - 784
#### Y son los valores esperados

In [4]:
Y

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [5]:
# reshape y conversion a int
Y = Y.reshape(m,1)

Y = (Y == np.array(range(10))).astype(int)

In [6]:
Y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0]])

#### Por la capacidad de la computadora, la capa por la que pasarán los datos de entrada tendrán 80 neuronas, por lo tanto lo valores de la matriz theta1 será (80,785)

#### La tercera capa será (10,81) y en ese orden para no utilizar la transpuesta

In [7]:
# random thetas
# theta1
theta1 = np.random.rand(80,785)
theta1 = theta1 / 1000
theta1

array([[7.06713277e-04, 7.74894427e-04, 7.79419426e-04, ...,
        8.49005195e-04, 5.47074107e-04, 7.62369168e-04],
       [7.03531332e-04, 8.00064859e-04, 5.18149662e-04, ...,
        1.12995270e-04, 7.17954970e-04, 4.09532716e-04],
       [7.05418511e-04, 6.01643209e-04, 5.38692265e-04, ...,
        1.51009518e-05, 9.34071278e-04, 3.63592406e-04],
       ...,
       [1.25022073e-04, 7.22372813e-04, 3.35671213e-04, ...,
        8.12617066e-04, 2.87881113e-04, 1.69036844e-04],
       [3.52342743e-05, 2.03027604e-04, 2.05054845e-04, ...,
        6.53933638e-04, 9.73498510e-06, 9.43162188e-04],
       [5.67635595e-04, 3.43364465e-04, 1.03181217e-05, ...,
        1.93577583e-04, 8.07709641e-04, 8.22999759e-04]])

In [8]:
# theta2
theta2 = np.random.rand(10, 81)
theta2 = theta2 / 1000
theta2

array([[3.72041138e-04, 1.39816182e-04, 8.06084585e-04, 4.30607022e-04,
        7.89420315e-04, 1.44469087e-04, 2.41111151e-04, 5.93035202e-04,
        7.04496516e-04, 1.20739757e-04, 8.73444907e-04, 7.03400206e-04,
        7.45010788e-04, 4.52158618e-05, 1.61792260e-04, 3.28217764e-04,
        1.78462688e-04, 9.10108203e-04, 3.33356296e-04, 4.26376234e-04,
        9.19313942e-04, 6.31421172e-04, 5.31716093e-05, 5.06240168e-04,
        5.49109806e-04, 5.61941469e-04, 8.81745992e-04, 4.55193634e-04,
        2.83420029e-06, 5.96979069e-04, 4.21663545e-04, 3.85861554e-04,
        9.42729519e-04, 4.85779474e-04, 7.81619657e-04, 7.66299103e-04,
        8.22030909e-04, 8.92209273e-04, 3.18867188e-04, 9.33041691e-04,
        8.26872814e-04, 1.95488867e-04, 8.14710705e-05, 4.73947674e-04,
        5.83270922e-04, 2.99145498e-04, 5.37459691e-04, 7.62840018e-04,
        6.36915454e-04, 6.81063583e-04, 8.22774646e-04, 6.33299564e-04,
        1.79791112e-04, 5.33360499e-04, 8.65792554e-05, 5.906288

#### Se realizó la división de 1000 de los valores de theta ya que al relizar las iteraciones de optimización se quedaba en la iteración 15

In [9]:
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

In [10]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

La sigmoide es la función de activación de "z" hacia "a"

In [11]:
def sigmoind(A):
    for cell in np.nditer(A, op_flags=['readwrite']):
        cell[...] = 1.0 / (1.0 + 2.718281828459045**(-1.0 * cell))
    return A

#### Feed forward, incluye la agregacion de la columna de unos en X y las dos transformaciones de a. Es el calculo hacia adelante de las "z" y "a" de 1 a la L


In [12]:
def feed_forward(thetas, X):
    a = [X]
    for i in range(len(thetas)):
        a.append(
            sigmoind(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                        a[i]
                    )),
                    thetas[i].T
                )
            )
        )
    return a


Calculamos delta super L. 

Ahora vamos a calcuar delta L para las capas (sin incluir la capa de entrada y final).

In [13]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

     
    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

In [14]:
def back_propagation(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    a = feed_forward(thetas, X)
    deltas = [*range(layers - 1), a[-1] - Y]

    # 2.4
    # se mueve desde L-1 porque no se debe tomar la ultima capa
    # tampoco se toma en cuenta la primera capa porque es la capa de entrada
    # no se toma la capa 1 ni 3, solo la 2
    for i in range(layers - 2, 0, -1):
        # NO TOMAR EN CUENTA LA PRIMERA COLUMNA de la matriz thetas[i] por bias.
        bias = np.delete(thetas[i], 0, 1)
        # se multiplica el producto de theta de la capa i por el error de la siguiente capa y el proucto de la salida de la capa i (a[i])  por 1 - a[i]
        deltas[i] = deltas[i + 1] @ bias * ( a[i] * ( 1-a[i]) )
    
    # se entrega una lista de los vectores columnas
    matrices_de_error = []

    # se está en 2.5
    # se mueve de 0 a L - 1
    for i in range(layers - 1):
        a[i] = np.hstack((np.ones(len(a[i])).reshape(len(a[i]), 1), a[i]))
        matrices_de_error.append(
            (deltas[i + 1].T @ a[i])/m
        )
    
    # regresa los theta aplastados para utilizarse en la optimizacion, esta concatenada de forma plana
    return flatten_list_of_arrays(np.asarray(matrices_de_error))

#### Agregar el theta1 (80,785) y theta2 (10,81) a una lista, esta se aplanará y se utilizará en el descenso de gradiente

In [15]:
arreglo_c = []
arreglo_c.append(theta1)
arreglo_c.append(theta2)

# flat_theta
flat_theta = flatten_list_of_arrays(arreglo_c)
flat_theta

array([0.00070671, 0.00077489, 0.00077942, ..., 0.00046716, 0.00038864,
       0.00032235])

In [16]:
# separar en shapes
NETWORK_ARCHITECTURE = np.array([
    n,
    80,
    10
])

theta_shapes = np.hstack((
    NETWORK_ARCHITECTURE[1:].reshape(len(NETWORK_ARCHITECTURE) - 1, 1),
    (NETWORK_ARCHITECTURE[:-1] + 1).reshape(len(NETWORK_ARCHITECTURE) - 1, 1)
))

In [17]:
"""print("OPTIMIZING...")
result = op.minimize(
    fun = cost_function,
    x0 = flat_theta,
    args = (theta_shapes, X, Y),
    method = 'L-BFGS-B',
    jac = back_propagation,
    options = {'disp': True, 'maxiter': 100}
)
print("OPTIMIZED")
print(result.x)"""

'print("OPTIMIZING...")\nresult = op.minimize(\n    fun = cost_function,\n    x0 = flat_theta,\n    args = (theta_shapes, X, Y),\n    method = \'L-BFGS-B\',\n    jac = back_propagation,\n    options = {\'disp\': True, \'maxiter\': 100}\n)\nprint("OPTIMIZED")\nprint(result.x)'

In [18]:
"""outfile = open("thetaOptimizado.txt", "wb")
pickle.dump(result.x, outfile)
outfile.close()"""

'outfile = open("thetaOptimizado.txt", "wb")\npickle.dump(result.x, outfile)\noutfile.close()'

Se realizó el método de optimización con 3000 iteraciones y se guardó en un archivo .txt, adjunto se encuentra un screenshot de cuando se terminó la optimización

<img src="screenshot.jpg"/>

In [19]:
def cost_functionO(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    
    # regresa la capa que tiene 3 neuronas
    return a[-1]

se desencripta el archivo .txt y se aplana

In [21]:
with open("thetaOptimizado1.txt", "rb") as openfile:
    while True:
        try:
            tOptimizado = pickle.load(openfile)
        except EOFError:
            break

flat_theta_optimizado = flatten_list_of_arrays(tOptimizado)

costoO = cost_functionO(flat_theta_optimizado, theta_shapes, X, Y)

f:\python364bits\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Se crea un contador y se va a hacer una interación de cada linea del array por medio del ciclo for
Es importante mencionar que se utiliza np.asarray para facilitar la búsqueda en columnas.

Para cada linea, se busca cual columna tiene el mayor argumento y se extrae el índice de la columna.
se compara el indice de la columna con la posición en Y[k] (que posee los valores esperados) y si ambos concuerdan, se aumenta el contador

In [22]:
contador_correctos = 0

for k in range(0,60000):
    max_index_col = np.argmax(np.asarray(costoO[k]), axis=0)
    #print(max_index_col)
    if np.argmax(np.asarray(Y[k]), axis=0) == max_index_col:
        #print("correcto :D")
        contador_correctos = contador_correctos + 1

print("Porcentaje: ", (contador_correctos * 100) / 60000)


Porcentaje:  83.48666666666666


# Porcentaje

In [23]:
(contador_correctos * 100) / 60000

83.48666666666666